In [ ]:
# @title Copyright & License (click to expand)
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的E2E ML：MLOps阶段7：监控：适用于自定义表格模型的Vertex AI模型监控

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage7/get_started_with_model_monitoring_custom.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage7/get_started_with_model_monitoring_custom.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td> 
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage7/get_started_with_model_monitoring_custom.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>
</table>

## 概述

本教程演示了如何为自定义表格模型使用Vertex AI模型监控。

### 目标

在这个笔记本中，你将学习如何使用`Vertex AI Model Monitoring`服务来检测自定义表格模型中输入预测请求中的特征偏差和漂移。

本教程使用以下 Google Cloud ML 服务：

- `Vertex AI Model Monitoring`
- `Vertex AI Prediction`
- `Vertex AI Model` 资源
- `Vertex AI Endpoint` 资源

执行的步骤包括：

- 下载一个预训练的自定义表格模型。
- 将预训练模型上传为`Model`资源。
- 将`Model`资源部署到`Endpoint`资源。
- 配置`Endpoint`资源进行模型监控。
- 生成用于特征偏差的合成预测请求。
- 等待电子邮件警报通知。
- 生成用于漂移的合成预测请求。
- 等待电子邮件警报通知。

了解更多关于 [Introduction to Vertex AI Model Monitoring](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview)。

### 模型

本教程使用一个预训练的模型，其模型工件存储在公共云存储桶中。

该模型基于[博客文章](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)。该模型背后的理念是您的公司拥有描述玩家如何与网站进行互动的海量日志数据。原始数据包含以下类别的信息：

- 身份 - 独特的玩家身份号码
- 人口统计特征 - 关于玩家的信息，如玩家所在地理区域
- 行为特征 - 玩家触发某些游戏事件的次数，比如达到新的等级
- 流失倾向 - 这是标签或目标特征，它提供了玩家可能流失的估计概率，即停止成为活跃玩家。

### 费用

本教程使用 Google Cloud 的可计费组件：

* Vertex AI
* BigQuery
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

### 建立您的本地开发环境

**如果您正在使用 Colab 或 Vertex AI Workbench 笔记本**，您的环境已经符合运行此笔记本的所有要求。您可以跳过此步骤。

**否则**，请确保您的环境符合此笔记本的要求。
您需要以下内容：

* Google Cloud SDK
* Git
* Python 3
* virtualenv
* 在使用 Python 3 的虚拟环境中运行 Jupyter Notebook

Google Cloud [设置Python开发环境](https://cloud.google.com/python/setup) 和[Jupyter安装指南](https://jupyter.org/install) 提供了满足这些要求的详细说明。以下步骤提供了一个简明的一套指导：

1. [安装并初始化 Cloud SDK。](https://cloud.google.com/sdk/docs/)

2. [安装 Python 3。](https://cloud.google.com/python/setup#installing_python)

3. [安装 virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) 并创建一个使用 Python 3 的虚拟环境。激活虚拟环境。

4. 要安装 Jupyter，请在终端窗口中的命令行上运行 `pip install jupyter`。

5. 要启动 Jupyter，请在终端窗口中的命令行上运行 `jupyter notebook`。

6. 在 Jupyter Notebook 仪表板中打开此笔记本。

## 安装

安装执行此笔记本所需的包。

In [ ]:
import os
import sys

assert sys.version_info.major == 3, "This notebook requires Python 3."

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# Install required packages.
! pip3 install {USER_FLAG} --quiet --upgrade google-cloud-aiplatform \
                                             google-cloud-bigquery

### 重新启动内核

安装完SDK后，您需要重新启动笔记本内核，以便它可以找到这些软件包。您可以通过 *Kernel -> 重新启动内核* 来重新启动内核，或者运行以下命令：

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 准备开始之前

### 设置您的谷歌云项目

**无论您使用什么笔记本环境，下面的步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建账号时，您会获得$300的免费信用用于支付计算/存储成本。

2. [确保为您的项目开启计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API 和 Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目ID。然后运行单元格以确保
Cloud SDK在这个笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter会运行以`!`为前缀的行作为shell命令，并将以`$`为前缀的Python变量插入这些命令中。

#### 设置你的项目ID

**如果你不知道你的项目ID**，你可以使用 `gcloud` 来获取你的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您也可以更改`REGION`变量，在本笔记本的其余部分中使用。以下是Vertex AI支持的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

**对于这个笔记本，我们建议您将区域设置为默认值`us-central1`**。

您不能使用多区域存储桶进行Vertex AI训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果您正在进行现场教程会话，则可能正在使用共享测试帐户或项目。为了避免用户在创建的资源之间的名称冲突，您应为每个实例会话创建一个UUID，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

#### 用户邮件
设置您的用户邮件地址以接收监控警报。

In [ ]:
USER_EMAIL = "[your-email-address]"  # @param {type:"string"}

### 验证您的Google Cloud帐户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经经过验证。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行身份验证以通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

1. 在Cloud Console中，转到[**创建服务帐号密钥**页](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务帐号**。

3. 在**服务帐号名称**字段中输入名称，然后点击**创建**。

4. 在**授予此服务帐号对项目的访问权限**部分，点击**角色**下拉列表。在过滤框中键入"Vertex AI"，并选择**Vertex AI管理员**。在过滤框中键入"存储对象管理员"，并选择**存储对象管理员**。

5. 点击**创建**。一个包含您密钥的JSON文件将下载到您的本地环境。

6. 在下面的单元格中将您的服务帐号密钥的路径作为`GOOGLE_APPLICATION_CREDENTIALS`变量输入，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 关于服务账户和权限的注意事项

**默认情况下不需要进行任何配置**，如果遇到任何权限相关的问题，请确保上述的服务账户拥有必要的角色：

|服务账户邮箱|描述|角色|
|---|---|---|
|PROJECT_NUMBER-compute@developer.gserviceaccount.com|Compute Engine 默认服务账户|Dataflow 管理员，Dataflow 工作者，存储管理员，BigQuery 管理员，Vertex AI 用户|
|service-PROJECT_NUMBER@gcp-sa-aiplatform.iam.gserviceaccount.com|AI 平台服务代理|Vertex AI 服务代理|

1. 前往https://console.cloud.google.com/iam-admin/iam。
2. 勾选“包括谷歌提供的角色授予”复选框。
3. 找到上述的邮箱。
4. 授予相应的角色。

### 使用来自不同项目的数据源
- 对于BQ数据源，请为两个服务账户授予“BigQuery 数据查看器”角色。
- 对于CSV数据源，请为两个服务账户授予“存储对象查看器”角色。

### 创建一个云存储桶

**无论你使用什么笔记本环境，都需要按照以下步骤操作。**

在下面设置你的云存储桶的名称，在本教程中你将使用它来上传监控服务的`输入架构`。

存储桶名称必须在所有谷歌云项目中全局唯一，包括组织外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = "gs://" + BUCKET_NAME

只有当您的存储桶不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查其内容验证对云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 导入库

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery
from google.cloud.aiplatform import model_monitoring

### 为 Python 初始化 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

在本教程中，您将使用与用于训练预训练模型的相同的公共BigQuery表中的数据。您将创建一个客户端接口，随后用于访问数据。

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

设置硬件加速器

您可以为预测（例如，GPU）设置硬件加速器，或选择不使用任何硬件加速器（CPU）。硬件加速器可以降低预测请求的延迟响应。在选择硬件加速器时，请考虑额外成本与延迟之间的权衡。

将变量`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持 GPU 的容器映像以及为虚拟机（VM）实例分配的 GPU 数量。例如，要使用一个 GPU 容器映像，并为每个 VM 分配 4 个 Nvidia Tesla K80 GPU，您需要指定：

（aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80，4）

请参阅[可用加速器的位置](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

否则，请指定`(None, None)`以使用一个在 CPU 上运行的容器映像。

In [ ]:
GPU = False
if GPU:
    DEPLOY_GPU, DEPLOY_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

设置预构建的Docker容器图像用于预测。

要查看最新列表，请访问[预构建的预测容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if GPU:
    DEPLOY_VERSION = "tf2-gpu.2-5"
else:
    DEPLOY_VERSION = "tf2-cpu.2-5"

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存
     - `n1-highmem`：每个vCPU 6.5GB内存
     - `n1-highcpu`：每个vCPU 0.9GB内存
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96\]

*注意：您也可以在训练和部署中使用n2和e2机器类型，但它们不支持GPU*。

In [ ]:
MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

## 介绍顶点AI模型监控

顶点AI模型监控支持AutoML表格模型和自定义表格模型。您可以监控入站预测请求中特征的偏差和漂移检测，也可以监控出站预测响应中特征归因（可解释AI）的偏差和漂移检测--即特征归因对输出（预测）的贡献分布。

以下是启用模型监控的基本步骤：

1. 将`顶点AI` AutoML或自定义表格模型部署到` Vertex AI Endpoint`。
2. 配置模型监控规范。
3. 将模型监控规范上传到`顶点AI Endpoint`。
4. 上传或自动生成`输入模式`以进行解析。
5. 对于特征偏差检测，上传训练数据以自动生成特征分布。
6. 对于特征归因，上传相应的`Vertex AI可解释性`规范。

配置完成后，您可以启用/禁用监控，更改警报并更新模型监控配置。

启用模型监控后，采样的传入预测请求将被记录到BigQuery表中。然后，对记录的请求中包含的输入特征值进行分析，以检查其是否存在倾斜或漂移，检查间隔由指定的时间间隔进行。您可以设置采样率以监视模型的生产输入的子集，以及监视间隔。

模型监控服务需要知道如何解析特征值，这称为输入模式。对于AutoML表格模型，输入模式会自动生成。对于自定义表格模型，服务将尝试从前1000个预测请求中自动生成输入模式。或者，您可以上传输入模式。

对于偏差检测，监控服务需要训练数据中值的统计分布基线。对于AutoML表格模型，这是自动生成的。对于自定义表格模型，您需要上传训练数据到服务，并让服务自动推导分布。

对于特征归因偏差和漂移检测，需要为自定义表格模型启用`Vertex AI可解释性`。对于AutoML模型，`Vertex AI可解释性`会自动启用。

了解更多关于[顶点AI模型监控介绍](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview)。

### 将模型工件上传为`Vertex AI Model`资源

首先，您可以使用`upload()`方法上传预训练的自定义表格模型工件作为`Vertex AI Model`资源，具有以下参数：

- `display_name`：`Model`资源的可读名称。
- `artifact_uri`：模型工件的云存储位置。
- `serving_container_image`：将模型部署到`Vertex AI Endpoint`资源时要使用的服务容器镜像。
- `sync`：是否等待进程完成，或立即返回（异步）。

In [ ]:
MODEL_ARTIFACT_URI = "gs://mco-mm/churn"

model = aiplatform.Model.upload(
    display_name="churn_" + UUID,
    artifact_uri=MODEL_ARTIFACT_URI,
    serving_container_image_uri=DEPLOY_IMAGE,
    sync=True,
)

print(model)

### 将`Vertex AI Model`资源部署到`Vertex AI Endpoint`资源

接下来，您可以使用`deploy()`方法将您的`Vertex AI Model`资源部署到`Vertex AI Endpoint`资源，使用以下参数：

- `deploy_model_display`：部署模型的人类可读名称。
- `machine_type`：每个VM节点实例的机器类型。
- `min_replica_count`：为自动扩展而提供的节点的最小数量。
- `max_replica_count`：为自动扩展而提供的节点的最大数量。
- `accelerator_type`：每个提供的节点的GPU加速器的类型（如果有）。
- `accelerator_count`：每个提供的节点的GPU加速器的数量（如果有）。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

if GPU:
    endpoint = model.deploy(
        deployed_model_display_name="churn_" + UUID,
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
        accelerator_type=DEPLOY_GPU.name,
        accelerator_count=DEPLOY_NGPU,
    )
else:
    endpoint = model.deploy(
        deployed_model_display_name="churn_" + UUID,
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

## 配置监控任务

配置监控任务包括以下规格：

- `alert_config`：发送监控警报的电子邮件地址。
- `schedule_config`：分析预测的时间窗口。
- `logging_sampling_strategy`：采样预测请求的速率。
- `drift_config`：要监控的特征和漂移阈值。
- `skew_config`：要监控的特征和偏斜阈值。

### 配置警报规范

首先，您可以使用以下设置配置`alerting_config`规范:

- `user_emails`: 一个或多个要发送警报的电子邮件列表。
- `enable_logging`: 将检测到的异常流式传输到Cloud Logging。默认为False。

In [ ]:
# Create alerting configuration.
alerting_config = model_monitoring.EmailAlertConfig(
    user_emails=[USER_EMAIL], enable_logging=True
)

### 配置监控间隔规范

接下来，您需要使用以下设置配置`schedule_config`规范：

- `monitor_interval`:设置模型监控作业的调度间隔，单位为小时。最小时间间隔为1小时。

In [ ]:
# Monitoring Interval
MONITOR_INTERVAL = 1  # @param {type:"number"}

# Create schedule configuration
schedule_config = model_monitoring.ScheduleConfig(monitor_interval=MONITOR_INTERVAL)

### 配置采样规范

接下来，您可以使用以下设置配置`logging_sampling_strategy`规范：

- `sample_rate`：作为百分比（介于0和1之间）的速率，用于随机采样监控中的预测请求。所选样本将被记录到BigQuery表中。

In [ ]:
# Sampling rate (optional, default=.8)
SAMPLE_RATE = 0.5  # @param {type:"number"}

# Create sampling configuration
logging_sampling_strategy = model_monitoring.RandomSampleConfig(sample_rate=SAMPLE_RATE)

### 配置漂移检测规范

接下来，您可以使用以下设置配置`drift_config`规范：

- `drift_thresholds`：一个键/值对的字典，其中键是用于监视漂移的输入特征。 值是检测阈值。 如果未指定，默认漂移特征的阈值为0.3（30％）。

*注意：* 启用漂移检测是可选的。

In [ ]:
DRIFT_THRESHOLD_VALUE = 0.05

DRIFT_THRESHOLDS = {
    "country": DRIFT_THRESHOLD_VALUE,
    "cnt_user_engagement": DRIFT_THRESHOLD_VALUE,
}

drift_config = model_monitoring.DriftDetectionConfig(drift_thresholds=DRIFT_THRESHOLDS)

### 配置偏斜检测规范

接下来，您将使用以下设置配置 `skew_config` 规范：

- `data_source`：原始训练数据集的数据源。数据源的格式默认为 BigQuery 表。否则，必须将设置 `data_format`设置为以下值之一。数据的位置必须是云存储位置。
  - `csv`： 
  - `jsonl`：
  - `tf-record`：
- `skew_thresholds`：一个键/值对字典，其中键是要监视偏斜的输入特征。值是检测阈值。如果未指定，特征的默认偏斜阈值为 0.3（30%）。
- `target_field`：训练数据集的目标标签

*备注：* 启用偏斜检测是可选的。

In [ ]:
# URI to training dataset.
DATASET_BQ_URI = "bq://mco-mm.bqmlga4.train"  # @param {type:"string"}
# Prediction target column name in training dataset.
TARGET = "churned"

SKEW_THRESHOLD_VALUE = 0.5

SKEW_THRESHOLDS = {
    "country": SKEW_THRESHOLD_VALUE,
    "cnt_user_engagement": SKEW_THRESHOLD_VALUE,
}

skew_config = model_monitoring.SkewDetectionConfig(
    data_source=DATASET_BQ_URI, skew_thresholds=SKEW_THRESHOLDS, target_field=TARGET
)

### 组装目标规范

最后，您将使用以下设置组装目标规范`objective_config`：

- `skew_detection_config`：（可选）倾斜检测配置的规范。
- `drift_detection_config`：（可选）漂移检测配置的规范。
- `explanation_config`：（可选）在启用特征归因监控时解释的规范。

In [ ]:
objective_config = model_monitoring.ObjectiveConfig(
    skew_detection_config=skew_config,
    drift_detection_config=drift_config,
    explanation_config=None,
)

### 创建输入模式

监控服务需要了解模型的特征和数据类型的输入特征，这被称为`输入模式`。`输入模式`可以通过以下方式之一来实现：
- 预加载到监控服务中。
- 在接收到前1000个预测实例后，由监控服务自动生成。

在本教程中，您将预加载`输入模式`。

#### 创建预定义的输入模式

预定义的`输入模式`是指定为YAML文件的。在此示例中，您将检索用于训练数据的BigQuery模式，其中包括特征名称和数据类型，以生成YAML规范。预定义的`输入模式`必须加载到Cloud Storage位置。

了解有关[用于解析输入的自定义实例模式](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview#custom-input-schemas)的更多信息。

In [ ]:
# Get the BQ table

table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])
bq_table = bqclient.get_table(table)

yaml = """type: object
properties:
"""

schema = bq_table.schema
for feature in schema:
    if feature.name == TARGET:
        continue
    if feature.field_type == "STRING":
        f_type = "string"
    else:
        f_type = "integer"
    yaml += f"""  {feature.name}:
    type: {f_type}
"""

yaml += """required:
"""
for feature in schema:
    if feature.name == TARGET:
        continue
    yaml += f"""- {feature.name}
"""

print(yaml)

with open("schema.yaml", "w") as f:
    f.write(yaml)

! gsutil cp schema.yaml {BUCKET_URI}/schema.yaml

### 创建监控作业

您可以使用`aiplatform.ModelDeploymentMonitoringJob.create()`方法，根据您的监控规范，创建一个监控作业，指定以下参数：

- `display_name`：监控作业的可读名称。
- `project`：项目ID。
- `region`：地区。
- `endpoint`：要启用监控的`Vertex AI Endpoint`的完全限定资源名称。
- `logging_sampling_strategy`：采样配置的规范。
- `schedule_config`：调度配置的规范。
- `alert_config`：警报配置的规范。
- `objective_configs`：目标配置的规范。
- `analysis_instance_schema_uri`：包含`输入模式`的YAML文件的位置。

In [ ]:
monitoring_job = aiplatform.ModelDeploymentMonitoringJob.create(
    display_name="churn_" + UUID,
    project=PROJECT_ID,
    location=REGION,
    endpoint=endpoint,
    logging_sampling_strategy=logging_sampling_strategy,
    schedule_config=schedule_config,
    alert_config=alerting_config,
    objective_configs=objective_config,
    analysis_instance_schema_uri=f"{BUCKET_URI}/schema.yaml",
)

print(monitoring_job)

#### 监控作业的电子邮件通知。

在警报配置中提供的电子邮件地址会收到一封电子邮件通知，通知模型监控作业现已启用。

内容如下：

您好，Vertex AI客户，

您收到此邮件是因为您正在使用Vertex AI模型监控服务。
此邮件是为了通知您，我们已收到您的请求，为下面列出的预测端点设置漂移或偏差检测。从现在开始，传入的预测请求将被抽样并记录以供分析。
原始请求和响应将从预测服务中收集，并保存在bq：//[your-project-id].model_deployment_monitoring_[endpoint-id].serving_predict中。

#### 监控工作状态

启动 `Vertex AI Model Monitoring` 作业后，直到计算出 `偏斜分布基线` 为止，它将处于 `待定` 状态。监控服务将启动一个批处理作业，从训练数据中生成分布基线。

一旦生成了基线分布，监控作业将进入 `离线` 状态。按照每个时间间隔的基础，例如每小时一次，监控作业在分析采样数据时会进入 `运行` 状态。分析完成后，它将返回到 `离线` 状态，等待下一次计划的分析。

In [ ]:
jobs = monitoring_job.list(filter=f"display_name=churn_{UUID}")
job = jobs[0]
print(job.state)

### 基准分布的自动生成

接下来，监控服务将创建一个批处理作业来分析训练数据，以生成基准分布。一旦完成，监控服务将开始按指定的间隔进行监视。

In [ ]:
import time

# Pause a bit for the baseline distribution to be calculated
if os.getenv("IS_TESTING"):
    time.sleep(180)

### 生成用于倾斜检测的合成预测请求

接下来，您从BigQuery培训表中提取前1000个实例以用于预测请求。您修改数据（合成数据）以触发预测请求中的倾斜检测，即训练分布与服务分布之间的差异如下：

- `country`：将所有值设置为加拿大

In [ ]:
# Download the table.
table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])

rows = bqclient.list_rows(table, max_results=1000)

instances = []
for row in rows:
    instance = {}
    for key, value in row.items():
        if key == TARGET:
            continue
        if value is None:
            value = ""
        if key == "country":
            value = "Canada"
        instance[key] = value
    instances.append(instance)

print(len(instances))

### 发出预测请求

接下来，使用`predict()`方法向您的`Vertex AI Endpoint`资源发送1000个预测请求。

In [ ]:
for instance in instances:
    response = endpoint.predict(instances=[instance])

prediction = response[0]

# print the prediction for the first instance
print(prediction[0])

### 记录抽样请求

一旦监控服务开始运行，抽样预测请求将被记录到云存储中。在下一个监控间隔期间，抽样预测将被复制到BigQuery日志表中。一旦条目在BigQuery表中，监控服务将分析抽样数据。

接下来，您需要等待第一个记录的条目出现在用于记录预测样本的BigQuery表中。由于您发送了1000个预测请求，采样率为50%，您应该看到大约500条记录。

In [ ]:
while True:
    time.sleep(180)

    ENDPOINT_ID = endpoint.resource_name.split("/")[-1]

    table = bigquery.TableReference.from_string(
        f"{PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"
    )
    rows = bqclient.list_rows(table)
    print(rows.total_rows)
    if rows.total_rows > 0:
        break

### 在监视过程中进行偏斜检测

特征输入偏斜检测将在下一个监控间隔发生。在本教程中，您将将监控间隔设置为一小时。因此，大约一个小时后，您的监视作业将从“离线”状态变为“运行”状态。在运行时，它将分析在此间隔内预测中记录的采样表，并将其与基准分布进行比较。

分析完成后，监视作业将通过电子邮件通知检测到的偏斜，本例中为“国家”，监视作业将进入“离线”状态，直到下一个间隔。

#### 等待监视间隔

从监视间隔上的分析发生到收到电子邮件警报，可能需要高达40分钟。

邮件内容将如下所示：

<blockquote>
   你好，Vertex AI客户，

您收到此邮件是因为您正在订阅Vertex AI模型监控服务。这封邮件仅仅是通知您，检测到了您部署的模型中的一些异常情况，可能需要您的关注。

基本信息:
终端名称: projects/[your-project-id]/locations/us-central1/endpoints/3315907167046860800
监视作业: projects/[your-project-id]/locations/us-central1/modelDeploymentMonitoringJobs/8672170640054157312
统计数据和异常根路径(Google Cloud存储): gs://cloud-ai-platform-773884b1-2a32-48d6-8b83-c03cde416b68/model_monitoring/job-8672170640054157312
BigQuery命令: SELECT * FROM `bq://[your-project-id].model_deployment_monitoring_3315907167046860800.serving_predict`

训练预测偏斜异常 (原始特征):
异常报告路径(Google Cloud存储): gs://cloud-ai-platform-773884b1-2a32-48d6-8b83-c03cde416b68/model_monitoring/job-8672170640054157312/serving/2022-08-25T00:00/stats_and_anomalies/<deployed-model-id>/anomalies/training_prediction_skew_anomalies

有关警报的更多信息，请访问模型监控警报页面。

部署的模型ID: <deployed-model-id>

特征名称	异常短描述	异常长描述
国家	训练和服务之间的高Linfty距离	Linfty距离在0.947563（高达六位有效数字）以上阈值0.5之上。具有最大差异的特征值为：加拿大
<blockquote>

In [ ]:
if os.getenv("IS_TESTING"):
    time.sleep(60 * 45)

### 生成用于漂移检测的合成预测请求

接下来，您从BigQuery训练表中提取相同的前1000个实例，用于预测请求。您修改数据（合成数据）以触发预测请求中的漂移检测，训练分布与服务分布如下：

- `cnt_user_engagement`：将值增加4倍。

In [ ]:
# Download the table.
table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])

rows = bqclient.list_rows(table, max_results=1000)

instances = []
for row in rows:
    instance = {}
    for key, value in row.items():
        if key == TARGET:
            continue
        if value is None:
            value = ""
        elif key == "cnt_user_engagement":
            value = int(value * 4)
        instance[key] = value
    instances.append(instance)

print(len(instances))

### 发送预测请求

接下来，您可以使用`predict()`方法将1000个预测请求发送到您的`Vertex AI Endpoint`资源。

In [ ]:
for instance in instances:
    response = endpoint.predict(instances=[instance])

prediction = response[0]

# print the prediction for the first instance
print(prediction[0])

### 记录取样请求

在接下来的监控间隔内，取样预测结果将被复制到BigQuery的日志表中。一旦这些条目进入BigQuery表中，监控服务将分析取样数据。

接下来，您等待第一个记录的条目在用于记录预测样本的BigQuery表中出现。由于您发送了1000个预测请求，采样率为50%，因此您应该看到大约1000个条目。

In [ ]:
while True:
    time.sleep(180)

    ENDPOINT_ID = endpoint.resource_name.split("/")[-1]

    table = bigquery.TableReference.from_string(
        f"{PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"
    )
    rows = bqclient.list_rows(table)
    print(rows.total_rows)
    if rows.total_rows > 550:
        break

### 监控中的漂移检测

特征输入漂移检测将在下一个监控间隔发生。在本教程中，您将监控间隔设置为一小时。因此，大约一个小时后，您的监控作业将从“离线”状态变为“运行”状态。在运行时，它将分析此间隔内预测的已记录采样表，并将其与先前监测间隔的分布进行比较。

分析完成后，监控作业将通过电子邮件通知检测到的漂移，本例中为`cnt_user_engagement`，并且监控作业将进入“离线”状态，直到下一个间隔。

#### 等待监控间隔

从分析所发生的监控间隔到收到电子邮件警报可能需要多达40分钟。

In [ ]:
if os.getenv("IS_TESTING"):
    time.sleep(60 * 45)

### 删除监控任务

您可以使用`delete()`方法来删除监控任务。

In [ ]:
monitoring_job.pause()
monitoring_job.delete()

#### 取消部署和删除`Vertex AI Endpoint`资源

您可以使用`delete()`方法删除您的`Vertex AI Endpoint`资源。在删除之前，必须先取消部署到您的`Vertex AI Endpoint`资源的任何模型。

In [ ]:
endpoint.undeploy_all()
endpoint.delete()

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以[删除您用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除此教程中创建的各个资源。

In [ ]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}

! rm -f schema.yaml

! bq rm -f {PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}